In [1]:
import pandas as pd, pymysql
from sshtunnel import SSHTunnelForwarder
from typing import Union

In [2]:
class MySQL :
    
    def __init__( self, db = '' ) -> None :
        self.db = db
        self.ssh : SSHTunnelForwarder
        self.con : pymysql.connections.Connection
        self.cur : pymysql.cursors.Cursor
        try :
            self.connect()
        except Exception as e :
            print( e )
        return
    
    def connect( self ) -> None :
        self.ssh = SSHTunnelForwarder(
            ssh_address_or_host = 'cu.epa.gov',
            ssh_username = 'zchiodin',
            ssh_password = '12Excelsiscelia$',
            remote_bind_address = ( 'ccte-mysql-res.epa.gov', 3306  )
            )
        self.ssh.start()
        self.con = pymysql.connect(
            user = 'zchiodini', 
            host = '127.0.0.1',
            password = 'o%Ff5ct4#gWo1&qD',
            database = self.db,
            port = self.ssh.local_bind_port,
            )
        self.cur = self.con.cursor()
        return
    
    def query( self, sql : str ) -> Union[ pd.DataFrame, None ] :
        '''Query MySQL'''
        try : 
            return pd.read_sql( sql, self.con, coerce_float = False )
        except Exception as e : 
            print( e )
        return
    
    def execute( self, sql : str, commit = False ) -> None :
        '''Execute script on MySQL'''
        try : 
            self.cur.execute( sql )
            if commit : 
                self.con.commit()
        except Exception as e :
            print( e )
        print( self.cur.rowcount, 'rows affected' )
        return
    
    def commit( self ) -> None :
        self.con.commit()
        return
    
    def close( self ) -> None :
        self.cur.close()
        self.con.close()
        self.ssh.close()
        self.ssh = ''
        self.con = ''
        return